In [1]:
import requests
import os
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from random import randint
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# CSV öffnen: 
# Hausbesitzer mit Koordinaten

haus = pd.read_csv('Hausbesitzer_mit_Koordinaten.csv', sep=';')
                     


In [3]:
# Sortieren nach Grundstück-Nummer
haus.sort_values("NameVorname").head(10)

# Zeilen mit NaN in Grundstueck-Nummer löschen
haus = haus.dropna(axis=0, subset=['Name'])

# Sortieren nach Grundstück-Nummer
haus.sort_values("NameVorname").head(10)


,Grundstueck-Nummer,EGID,Gebaeude-Koordinate X,Gebaeude-Koordinate Y,Gebaeude-Strasse,Gebaeude-Hausnummer,Gebaeude-PLZ,Gebaeude-Ort,Koordinate,Grundstueck-Typ,Grundstueck-Flaeche,Eigentumsart,Name,Vorname,NameVorname,Adresse,PLZ,Ort,Datum der Abfrage
4333,3041,001 738 647,2759745,1190820,Süsswinkelgasse,6,7000,Chur,POINT (2759745 1190820),Liegenschaft,290.0,Stockwerkeigentum,7402 Immobilien AG,NaN,7402 Immobilien AG,Industriestrasse 5,7402.0,Bonaduz,2019-12-29 02:45:12.816469
5359,1120,001 741 650,2758723,1191879,Albulastrasse,40,7000,Chur,POINT (2758723 1191879),Liegenschaft,2667.0,Gesamteigentum,7402 Immobilien AG,NaN,7402 Immobilien AG,Industriestrasse 5,7402,Bonaduz,2020-01-01 10:06:55.323045
10568,1120,001 741 648,2758722,1191879,Albulastrasse,38,7000,Chur,POINT (2758722 1191879),Liegenschaft,2667.0,Gesamteigentum,7402 Immobilien AG,NaN,7402 Immobilien AG,Industriestrasse 5,7402,Bonaduz,2020-01-01 10:06:55.323045
10939,11156,160 011 319,2758592,1191114,Segantinistrasse,44,7000,Chur,POINT (2758592 1191114),Liegenschaft,1181.0,Stockwerkeigentum,7402 Immobilien AG,NaN,7402 Immobilien AG,Industriestrasse 5,7402,Bonaduz,2019-12-30 14:25:58.441857
11132,11820,160 013 687,2758620,1191091,Segantinistrasse,42,7000,Chur,POINT (2758620 1191091),Liegenschaft,1180.0,Stockwerkeigentum,7402 Immobilien AG,NaN,7402 Immobilien AG,Industriestrasse 5,7402,Bonaduz,2019-12-30 15:38:57.302760
14133,3060,001 738 470,2759465,1190717,Untere Gasse,26,7000,Chur,POINT (2759465 1190717),Liegenschaft,169.0,Alleineigentum,7402 Immobilien AG,NaN,7402 Immobilien AG,Industriestrasse 5,7402,Bonaduz,2019-12-29 02:52:34.636383
8521,5232,191 400 450,2757880,1191515,Industriestrasse,5,7000,Chur,POINT (2757880 1191515),Liegenschaft,4847.0,Alleineigentum,A. Vögele & Söhne Recycling AG,NaN,A. Vögele & Söhne Recycling AG,Industriestrasse 5A,7000,Chur,2019-12-29 16:48:06.700414
8515,6459,009 001 016,2759606,1191268,Alexanderstrasse,18,7000,Chur,POINT (2759606 1191268),Liegenschaft,347.0,Stockwerkeigentum,AAA Immobilien Bösch AG,NaN,AAA Immobilien Bösch AG,Brunnmattstrasse 52,8103,Unterengstringen,2020-01-07 16:23:37.529989
9013,1014,001 740 746,2759477,1191777,Wiesentalstrasse,41,7000,Chur,POINT (2759477 1191777),Liegenschaft,4014.0,Alleineigentum,AKTIENGESELLSCHAFT EMMAWEG,NaN,AKTIENGESELLSCHAFT EMMAWEG,Quaderstrasse 11,7000,Chur,2019-12-28 17:34:10.345840
7483,1014,001 740 748,2759469,1191821,Wiesentalstrasse,45,7000,Chur,POINT (2759469 1191821),Liegenschaft,4014.0,Alleineigentum,AKTIENGESELLSCHAFT EMMAWEG,NaN,AKTIENGESELLSCHAFT EMMAWEG,Quaderstrasse 11,7000,Chur,2019-12-28 17:34:10.345840


In [4]:
haus['Grundstueck-Flaeche'].median()

1985.0

In [5]:
haus['Grundstueck-Flaeche'].max()

6591039.0

In [6]:
haus['Grundstueck-Flaeche'].min()

34.0

In [7]:
# die 50 grössten haus-Besitzer von Chur (Anzahl Häuser)

haus['NameVorname'].value_counts().head(50)

# -> Verzerrung wegen Stockwerkeigentum und gemeinsamem Eigentum bei Stockwerkeigentum

Stadt Chur                                158
Pensionskasse Graubünden                   54
Kanton Graubünden                          40
Zindel Gaudenz Stefan                      40
Febo Sandra Martina                        40
domenig IMMOBILIEN AG                      40
Fusar Poli-Zindel Flurina Gemma Olivia     40
Casal Johann Andrea                        36
Bürgergemeinde Chur                        36
Stäubli Roger                              35
Lendi Maria 'Elisabeth'                    35
Lendi Balz Heinz                           35
Meyer Pia 'Hyazintha'                      34
Meyer Urs David                            34
Ardüser Johannes                           34
Spescha Johann Georg                       33
Spescha Beatrice Cornelia                  33
Zindel Barfuss Karin Gabriela              33
Brechbühl Christoph                        33
Räschle Felix Paul                         33
Meier Edwin Alfred                         32
Dorta Romano Peder                

In [8]:
# die 50 grössten haus-Besitzer von Chur (Anzahl Häuser)

# NICHT Stockwerkeigentümer

ohne_stwe = haus['Eigentumsart'] != 'Stockwerkeigentum'

ohne_stwe = haus[ohne_stwe]

ohne_stwe.groupby('NameVorname')['Grundstueck-Nummer'].value_counts().sort_values(ascending=False).head(50)


NameVorname                                                  Grundstueck-Nummer
AXA Versicherungen AG                                        1869                  18
Bürgergemeinde Chur                                          3479                  14
Anlagestiftung der Migros-Pensionskasse Immobilien           11303                 12
Mensa episcopalis curiensis                                  2438                  11
Helvetia Schweizerische Lebensversicherungsgesellschaft AG   3996                  10
                                                             5060                   9
Schweizerische Eidgenossenschaft (VBS)                       1810                   9
Bürgergemeinde Chur                                          3476                   7
Stiftung Kantonsspital Graubünden                            1624                   7
Wohnbaugenossenschaft Am Salvatorenturm                      1812                   7
Schweizerische Eidgenossenschaft (EFZD)                     